In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd

from massbank2db.db import MassbankDB
from massbank2db.spectrum import MBSpectrum

In [3]:
MB_DBFN = "massbank__2020.11__v0.6.1.sqlite"
PC_DBFN = "/home/bach/Documents/doctoral/projects/local_pubchem_db/db_files/pubchem_01-02-2021.sqlite"

# Tutorial: Use a local copy of MassBank DB

This tutorial illustrates possible usecases of a local MassBank DB and how those are supported by the ```massbank2db``` package.

## Iterate over the Spectra of a Dataset

Here we illustrate how the spectra (or MassBank entries) of a particular dataset can be iterated over. This might be usefull, when for example the spectra (including their meta-information, such as the molecular structure and the retention time (RT)) should be used to train a machine learning algorithm. 

### Individual Spectra

In [7]:
# We connect to the MassBank DB constructed in the 'Tutorial__Build_DB.ipynb'
with MassbankDB(MB_DBFN) as mbdb: 
    for idx, (mol, spec, cands) in enumerate(mbdb.iter_spectra(dataset="AU_000", grouped=False)):
        
        # For illustration purposes, we only load the first 10 spectra
        if idx > 9:
            break
            
        print(spec.get("accession"), "mol=%s" % mol["inchikey"], "n_peaks=%d" % len(spec.get_mz()))

AU101851 mol=JLKIGFTWXXRPMT-UHFFFAOYSA-N n_peaks=2
AU101852 mol=JLKIGFTWXXRPMT-UHFFFAOYSA-N n_peaks=1
AU101853 mol=JLKIGFTWXXRPMT-UHFFFAOYSA-N n_peaks=1
AU106751 mol=NQPDXQQQCQDHHW-UHFFFAOYSA-N n_peaks=4
AU106752 mol=NQPDXQQQCQDHHW-UHFFFAOYSA-N n_peaks=6
AU106753 mol=NQPDXQQQCQDHHW-UHFFFAOYSA-N n_peaks=6
AU106754 mol=NQPDXQQQCQDHHW-UHFFFAOYSA-N n_peaks=5
AU106755 mol=NQPDXQQQCQDHHW-UHFFFAOYSA-N n_peaks=5
AU108451 mol=GOVWOKSKFSBNGD-UHFFFAOYSA-N n_peaks=8
AU108452 mol=GOVWOKSKFSBNGD-UHFFFAOYSA-N n_peaks=4


### Group spectra corresponding to the same compound and setting

MassBank contains spectra corresponding to the same compound, MS and LC configuration within one (contributor, accession-prefix)-tuple as separate files, e.g. if multiple collision energies where measured. In practice we can merge those spectra, for example to given them as input to a machine learning approach. The following SQLite query is used to group the spectra:
```SQLite
GROUP BY dataset, molecule, precursor_mz, precursor_type, fragmentation_mode
```
The dataset here also encodes the ionization mode, i.e. positive or negative ionization. The iterator can be modified to return the grouped spectra:

In [9]:
# We connect to the MassBank DB constructed in the 'Tutorial__Build_DB.ipynb'
with MassbankDB(MB_DBFN) as mbdb: 
    for idx, (mol, specs, cands) in enumerate(mbdb.iter_spectra(dataset="AU_000", grouped=True)):
        
        # For illustration purposes, we only load the first 10 spectra
        if idx > 9:
            break
            
        print([spec.get("accession") for spec in specs], "mol=%s" % mol["inchikey"], 
              "n_peaks=[%s]" % ",".join([str(len(spec.get_mz())) for spec in specs]))

['AU230657', 'AU230658', 'AU230659', 'AU230660', 'AU230662'] mol=WIIZWVCIJKGZOK-UHFFFAOYSA-N n_peaks=[19,12,6,3,10]
['AU238262'] mol=YGSDEFSMJLZEOE-UHFFFAOYSA-N n_peaks=[4]
['AU229857', 'AU229858'] mol=BTJIUGUIPKRLHP-UHFFFAOYSA-N n_peaks=[3,3]
['AU230257', 'AU230258', 'AU230259'] mol=UFBJCMHMOXMLKC-UHFFFAOYSA-N n_peaks=[4,7,5]
['AU229357', 'AU229358', 'AU229359', 'AU229360', 'AU229362'] mol=UQVKZNNCIHJZLS-UHFFFAOYSA-N n_peaks=[2,2,4,2,5]
['AU228110', 'AU228111', 'AU228112', 'AU228157', 'AU228158', 'AU228159'] mol=GOEMGAFJFRBGGG-UHFFFAOYSA-N n_peaks=[3,2,3,2,4,3]
['AU311151'] mol=YASYVMFAVPKPKE-UHFFFAOYSA-N n_peaks=[2]
['AU231057', 'AU231058', 'AU231059', 'AU231060'] mol=FZEYVTFCMJSGMP-UHFFFAOYSA-N n_peaks=[3,3,8,10]
['AU231357', 'AU231358', 'AU231359', 'AU231360', 'AU325851', 'AU325852', 'AU325853', 'AU325854'] mol=ZOMSMJKLGFBRBS-UHFFFAOYSA-N n_peaks=[7,18,18,10,3,6,4,2]
['AU234957', 'AU234958', 'AU234959', 'AU234960'] mol=LKJPYSCBVHEWIU-UHFFFAOYSA-N n_peaks=[8,9,13,8]


### Retrive molecular candidates for the spectra

For some applications it is useful to extract a molecular candidate for each spectrum. That is, a set of potential molecules, that could have been measured in the spectrum. There are two common approaches to get the set of molecular candidates:

1. ```return_candidates="mf"```: Using the ground-truth / predicted molecular formula of a spectrum
2. ```return_candidates="mz_gt"```: Using a mass-window around the ground truth monoisotopic mass of a spectrum
2. ```return_candidates="mz_measured"```: Using a mass-window around the calculated (from precursor type) monoisotopic mass of a spectrum

In [21]:
# We connect to the MassBank DB constructed in the 'Tutorial__Build_DB.ipynb'
with MassbankDB(MB_DBFN) as mbdb: 
    for idx, (mol, specs, cands) in enumerate(mbdb.iter_spectra(
        dataset="AU_000", grouped=True, return_candidates="mf", pc_dbfn=PC_DBFN)):
        
        # For illustration purposes, we only load the first 10 spectra
        if idx > 9:
            break
            
        print([spec.get("accession") for spec in specs], 
              "mol=%s" % mol["inchikey"],
              "n_peaks=[%s]" % ",".join([str(len(spec.get_mz())) for _spec in specs]), 
              "n_cand=%d" % len(cands),
              "rt=[%s]" % ",".join([str(_spec.get("retention_time")) for _spec in specs]))

print("\nCandidates are as Pandas DataFrame")
print(cands.head())

['AU230657', 'AU230658', 'AU230659', 'AU230660', 'AU230662'] mol=WIIZWVCIJKGZOK-UHFFFAOYSA-N n_peaks=[40,40,40,40,40] n_cand=98 rt=[5.69,5.677,5.699,5.668,5.677]
['AU238262'] mol=YGSDEFSMJLZEOE-UHFFFAOYSA-N n_peaks=[40] n_cand=429 rt=[3.637]
['AU229857', 'AU229858'] mol=BTJIUGUIPKRLHP-UHFFFAOYSA-N n_peaks=[40,40] n_cand=410 rt=[5.616,5.612]
['AU230257', 'AU230258', 'AU230259'] mol=UFBJCMHMOXMLKC-UHFFFAOYSA-N n_peaks=[40,40,40] n_cand=113 rt=[4.439,4.418,4.412]
['AU229357', 'AU229358', 'AU229359', 'AU229360', 'AU229362'] mol=UQVKZNNCIHJZLS-UHFFFAOYSA-N n_peaks=[40,40,40,40,40] n_cand=3995 rt=[7.251,7.244,7.253,7.241,7.265]
['AU228110', 'AU228111', 'AU228112', 'AU228157', 'AU228158', 'AU228159'] mol=GOEMGAFJFRBGGG-UHFFFAOYSA-N n_peaks=[40,40,40,40,40,40] n_cand=5657 rt=[4.973,4.966,4.98,4.982,4.997,4.98]
['AU311151'] mol=YASYVMFAVPKPKE-UHFFFAOYSA-N n_peaks=[40] n_cand=33 rt=[2.927]
['AU231057', 'AU231058', 'AU231059', 'AU231060'] mol=FZEYVTFCMJSGMP-UHFFFAOYSA-N n_peaks=[40,40,40,40] n_ca

## Merge Spectra

In the previous examples we have shown the support for spectra grouping. Sometimes it can be useful to merge the grouped spectra into one. For that, the peaks of all spectra are joint into one peak list. Subsequently, a hierarchical clustering is applied to group peaks that are belonging to the same fragment allowing a small mass-per-charge deviation. The retention times can also be merged, e.g. by taking the minimum retention time of all spectra. 

In [23]:
# We connect to the MassBank DB constructed in the 'Tutorial__Build_DB.ipynb'
with MassbankDB(MB_DBFN) as mbdb: 
    for idx, (mol, specs, cands) in enumerate(mbdb.iter_spectra(dataset="BML_000", grouped=True)):
        
        # For illustration purposes, we only load the first 10 spectra
        if idx > 9:
            break

        # Spectra peaks are merged into a single spectrum. 
        spec = MBSpectrum.merge_spectra(specs)
            
        print("NEW ID:", spec.get("accession"), "ORIGINAL IDs:", spec.get("original_accessions"), "mol=%s" % mol["inchikey"]) 
        print("\t", 
              "n_peaks (original)=[%s]" % ",".join([str(len(_spec.get_mz())) for _spec in specs]),
              "n_peaks (merged)=[%s]" % len(spec.get_mz()))
        print("\t",
              "RTs (original)=[%s]" % ",".join([str(_spec.get("retention_time")) for _spec in specs]), 
              "RTs (merged)=[%f]" % spec.get("retention_time"))

NEW ID: BML9840033 ORIGINAL IDs: ['BML00305', 'BML00309'] mol=YBJHBAHKTGYVGT-UHFFFAOYSA-N
	 n_peaks (original)=[11,37] n_peaks (merged)=[40]
	 RTs (original)=[3.46,3.46] RTs (merged)=[3.460000]
NEW ID: BML0035840 ORIGINAL IDs: ['BML00989', 'BML00997', 'BML01005'] mol=BHQCQFFYRZLCQQ-UHFFFAOYSA-N
	 n_peaks (original)=[6,7,24] n_peaks (merged)=[31]
	 RTs (original)=[10.065,10.065,10.065] RTs (merged)=[10.065000]
NEW ID: BML4708948 ORIGINAL IDs: ['BML01334'] mol=CKLJMWTZIZZHCS-UHFFFAOYSA-N
	 n_peaks (original)=[6] n_peaks (merged)=[6]
	 RTs (original)=[0.367] RTs (merged)=[0.367000]
NEW ID: BML8747603 ORIGINAL IDs: ['BML01204'] mol=QIAFMBKCNZACKA-UHFFFAOYSA-N
	 n_peaks (original)=[6] n_peaks (merged)=[6]
	 RTs (original)=[2.327] RTs (merged)=[2.327000]
NEW ID: BML5474079 ORIGINAL IDs: ['BML01273', 'BML01276', 'BML01279'] mol=CXQWRCVTCMQVQX-UHFFFAOYSA-N
	 n_peaks (original)=[65,63,52] n_peaks (merged)=[140]
	 RTs (original)=[4.365,4.365,4.365] RTs (merged)=[4.365000]
NEW ID: BML1677811 ORIG

## Convert Spectra into Format for InSilico Structure Identifyer

### MetFrag Format

The library allows you to output the spectra in the format needed as input for MetFrag. The files written to the ```./metfrag/``` can be directly used to run the [MetFrag scoring CLI](https://ipb-halle.github.io/MetFrag/projects/metfragcl/) by executing the following command: 
```bash
java -jar ~/.local/bin/MetFrag2.4.5.jar AU879682_config.txt
```

In [46]:
# We connect to the MassBank DB constructed in the 'Tutorial__Build_DB.ipynb'
with MassbankDB(MB_DBFN) as mbdb: 
    for idx, (mol, specs, cands) in enumerate(mbdb.iter_spectra(dataset="AU_000", grouped=True, return_candidates="mf", 
                                                                pc_dbfn=PC_DBFN)):
        
        # Spectra peaks are merged into a single spectrum. 
        spec = MBSpectrum.merge_spectra(specs)
        
        # Output the merged spectrum in MetFrag format
        metfrag_output = spec.to_metfrag_format(
            molecular_candidates=cands,
            **{"MetFragScoreWeights": [1.0],
               "MetFragScoreTypes": ["FragmenterScore"],
               "LocalDatabasePath": "./",
               "ResultsPath": "./",
               "NumberThreads": 4,
               "PeakListPath": "./",
               "UseSmiles": False
              }
        )
        
        for k, v in metfrag_output.items(): 
            print("============")
            print(k)
            print("------------")
            print(v)
        
            with open(os.path.join("metfrag", k), "r+") as ofile: 
                ofile.write(v)
        
        break

AU51150706.peaks
------------
45.992500	0.179649
82.946300	0.061754
121.029760	1.000000
122.031450	0.111597
122.036000	0.056842
148.038950	0.138950
151.027340	1.000000
152.034840	1.000000
153.037933	0.135667
159.028900	0.058246
159.034500	0.041058
166.015500	0.067368
176.034633	0.239209
177.040000	0.050424
179.046900	0.033952
194.045200	0.534893
195.049300	0.077854
219.038700	0.052037
221.054400	0.039129
237.055400	0.087939
249.050600	0.188786
257.033500	0.738604
258.037400	0.106091
259.027900	0.230738
321.003700	0.971359
322.007800	0.170633
323.002800	0.641791
AU51150706.cands
------------
MonoisotopicMass|InChI|Identifier|InChIKey|MolecularFormula|SMILES
322.012327|InChI=1S/C11H12Cl2N2O5/c1-5(16)19-9-6(4-12)20-10(8(9)13)15-3-2-7(17)14-11(15)18/h2-3,6,8-10H,4H2,1H3,(H,14,17,18)/t6-,8-,9-,10-/m1/s1|121221881|MCLTZAZBQCRJDE-PEBGCTIMSA-N|C11H12Cl2N2O5|CC(=O)O[C@@H]1[C@H](O[C@H]([C@@H]1Cl)N2C=CC(=O)NC2=O)CCl
322.012327|InChI=1S/C11H12Cl2N2O5/c1-6(11(17,18)14-10(16)9(12)13)7-2-4-8(5-3-7)15

### SIRIUS / CSI:FingerID

In [56]:
# We connect to the MassBank DB constructed in the 'Tutorial__Build_DB.ipynb'
with MassbankDB(MB_DBFN) as mbdb: 
    for idx, (mol, specs, cands) in enumerate(mbdb.iter_spectra(dataset="AU_000", grouped=True, return_candidates="mf", 
                                                                pc_dbfn=PC_DBFN)):
        
        # Spectra peaks are merged into a single spectrum. 
        spec = MBSpectrum.merge_spectra(specs, merge_peak_lists=False)
        
        # Output the merged spectrum in SIRIUS (.ms) format
        sirius_output = spec.to_sirius_format(molecular_candidates=cands, add_gt_molecular_formula=True)
        
        for k, v in sirius_output.items(): 
            print("============")
            print(k)
            print("------------")
            print(v)
        
            with open(os.path.join("sirius", k), "w") as ofile: 
                ofile.write(v)
        
        break

AU51150706.ms
------------
>compound AU51150706
>parentmass 321.0051
>ionization [M-H]-
>formula C11H12Cl2N2O5
>rt 340.932000
>profile qtof
#smiles_iso C1=CC(=CC=C1C(C(CO)NC(=O)C(Cl)Cl)O)[N+](=O)[O-]
#smiles_can C1=CC(=CC=C1C(C(CO)NC(=O)C(Cl)Cl)O)[N+](=O)[O-]
#inchikey WIIZWVCIJKGZOK-UHFFFAOYSA-N
#pubchem_id 298
#mb_accession AU230657,AU230658,AU230659,AU230660,AU230662

>ms2
121.029700 1008.000000
148.038200 360.000000
151.027200 3888.000000
152.035100 9916.000000
153.038300 760.000000
176.034400 2372.000000
177.040000 500.000000
194.045300 5304.000000
195.049300 772.000000
219.038700 516.000000
221.054400 388.000000
237.055400 872.000000
249.049500 1872.000000
257.033200 7324.000000
258.037400 1052.000000
259.027900 2288.000000
321.003700 9632.000000
322.007800 1692.000000
323.002800 6364.000000

>ms2
121.030200 2576.000000
122.030600 364.000000
148.039700 800.000000
151.027400 7020.000000
152.035100 10132.000000
153.038900 976.000000
159.034500 416.000000
176.035200 2008.000000
179.